In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from vtl.Readfile import Readfile
from glob import glob
from astropy.io import fits, ascii
from astropy.table import Table
from astropy import wcs
from scipy.interpolate import interp1d
import os
import cPickle
import seaborn as sea
sea.set(style='white')
sea.set(style='ticks')
sea.set_style({"xtick.direction": "in","ytick.direction": "in"})
colmap = sea.cubehelix_palette(12, start=2, rot=.2, dark=0, light=1.1, as_cmap=True)

In [3]:
ids,speclist,lmass,rshift,flxr=np.array(Readfile('masslist_jan26.dat',is_float=False))
lmass,rshift,flxr=np.array([lmass,rshift,flxr]).astype(float)

IDA=[]  # all masses in sample
IDL=[]  # low mass sample
IDH=[]  # high mass sample

for i in range(len(ids)):
    if 10.0<=lmass[i] and 1<rshift[i]<1.75:
        IDA.append(i)
    if 10.931>lmass[i] and 1<rshift[i]<1.75:
        IDL.append(i)
    if 10.931<lmass[i] and 1<rshift[i]<1.75:
        IDH.append(i)

metal=np.array([ 0.0020, 0.0025, 0.0031, 0.0039, 0.0049, 0.0061,  0.0068,  0.0077,  0.0085,  0.0096,  0.0106,
                  0.012, 0.0132, 0.014,  0.0150,  0.0164, 0.018,  0.019,  0.021,  0.024, 0.027, 0.03])
bcmetal=np.array([0.0001, 0.0004, 0.004, 0.008, 0.02])
age=np.arange(.5,6.1,.1)
tau=[0,8.0,8.15,8.28,8.43,8.57,8.72,8.86,9.0,9.14,9.29,9.43,9.57,9.71,9.86,10.0]

M,A=np.meshgrid(metal,age)
BM,BA=np.meshgrid(bcmetal,age)

In [4]:
fp = '/Users/Vince.ec/Clear_data/flt_files/'

flt_list=glob(fp + 'ic*flt.fits')

specflt=[]

for i in range(len(flt_list)):
    indat=fits.open(flt_list[i])
    if indat[0].header['OBSTYPE'] ==  'SPECTROSCOPIC':
        specflt.append(flt_list[i])

In [5]:
catn=Table.read('/Users/Vince.ec/Clear_data/goodsn_mosaic/goodsn_3dhstP.cat', format='ascii')
cats=Table.read('/Users/Vince.ec/Clear_data/goodss_mosaic/goodss_3dhst.v4.3.cat', format='ascii')

In [7]:
Ra=[]
Dec=[]

for i in range(len(ids)):
    if ids[i][0] == 'n':
        Ra.append(float(catn['ra'][np.argwhere(catn['id']==int(ids[i][1:]))]))
        Dec.append(float(catn['dec'][np.argwhere(catn['id']==int(ids[i][1:]))]))
    if ids[i][0] == 's':
        Ra.append(float(cats['ra'][np.argwhere(cats['id']==int(ids[i][1:]))]))
        Dec.append(float(cats['dec'][np.argwhere(cats['id']==int(ids[i][1:]))]))    

cat_frame=pd.DataFrame({'ids' : ids, 'ra' : Ra, 'dec' : Dec})
print cat_frame

          dec     ids          ra
0   62.214608  n14713  189.236333
1   62.218809  n15616  189.238403
2   62.224840  n16758  189.162357
3   62.227726  n17436  189.171078
4   62.237460  n19442  189.123036
5   62.247548  n21156  189.239409
6   62.247344  n21427  189.368121
7   62.256571  n23435  189.338306
8   62.309192  n32562  189.168890
9   62.307577  n32566  189.161280
10  62.317153  n33780  189.202025
11  62.323647  n34694  189.147840
12  62.329662  n35469  189.220600
13  62.360820  n37686  189.274474
14  62.377789  n38126  189.284502
15 -27.802087  s23421   53.137760
16 -27.785866  s26915   53.165159
17 -27.742385  s35774   53.158775
18 -27.739000  s36095   53.162610
19 -27.727300  s38785   53.168249
20 -27.727621  s39012   53.064240
21 -27.725868  s39170   53.041826
22 -27.726209  s39241   53.042327
23 -27.726461  s39364   53.062821
24 -27.725928  s39631   53.042169
25 -27.724640  s39804   53.178423
26 -27.724724  s39805   53.163237
27 -27.724832  s39825   53.132667
28 -27.719472 

In [11]:
def Source_present(fn,ra,dec):
    flt=fits.open(fn)
    w = wcs.WCS(flt[1].header)
    present=False

    xpixlim=len(flt[1].data[0])
    ypixlim=len(flt[1].data)

    [pos]=w.wcs_world2pix([[ra,dec]],1)



    if -100<pos[0]<xpixlim + 100 and -100<pos[1]<ypixlim + 100:
        present=True

    return present,pos

filelist=[]
for i in range(len(ids)):
    tpos=[]
    tfiles=[]
    for ii in range(len(specflt)):
        pres,pos=Source_present(specflt[ii],Ra[i],Dec[i])
        if pres==True:
            tpos.append(pos)
            tfiles.append(specflt[ii])
    mintpos=np.abs(np.array(tpos)-507)
    dist=np.sqrt(mintpos.T[0]**2+mintpos.T[1]**2)
    filelist.append(np.array(tfiles)[np.argmin(dist)])
    
print filelist

['/Users/Vince.ec/Clear_data/flt_files/icat12tqq_flt.fits', '/Users/Vince.ec/Clear_data/flt_files/icat12tqq_flt.fits', '/Users/Vince.ec/Clear_data/flt_files/icat08huq_flt.fits', '/Users/Vince.ec/Clear_data/flt_files/icat08hnq_flt.fits', '/Users/Vince.ec/Clear_data/flt_files/icxt40npq_flt.fits', '/Users/Vince.ec/Clear_data/flt_files/icxt55krq_flt.fits', '/Users/Vince.ec/Clear_data/flt_files/icat26dyq_flt.fits', '/Users/Vince.ec/Clear_data/flt_files/icat26dyq_flt.fits', '/Users/Vince.ec/Clear_data/flt_files/icxt43vwq_flt.fits', '/Users/Vince.ec/Clear_data/flt_files/icxt43vwq_flt.fits', '/Users/Vince.ec/Clear_data/flt_files/icat18kgq_flt.fits', '/Users/Vince.ec/Clear_data/flt_files/icxt42qbq_flt.fits', '/Users/Vince.ec/Clear_data/flt_files/icat18kgq_flt.fits', '/Users/Vince.ec/Clear_data/flt_files/icxt49hmq_flt.fits', '/Users/Vince.ec/Clear_data/flt_files/icxt47xfq_flt.fits', '/Users/Vince.ec/Clear_data/flt_files/icxt26hcq_flt.fits', '/Users/Vince.ec/Clear_data/flt_files/icxt28wbq_flt.fit

In [12]:
flt_source=pd.DataFrame({'spec_files' : speclist, 'flt' : filelist})
flt_source.to_csv('flt_w_sources.dat',sep=' ')

In [21]:
# sum(in_field)
totsum=sum(in_field)

for i in range(len(ids)):
    print totsum[i],ids[i]

1.0 n14713
1.0 n15616
4.0 n16758
2.0 n17436
2.0 n19442
4.0 n21156
0.0 n21427
4.0 n23435
4.0 n32562
3.0 n32566
2.0 n33780
3.0 n34694
2.0 n35469
3.0 n37686
1.0 n38126
3.0 s23421
3.0 s26915
3.0 s35774
3.0 s36095
3.0 s38785
3.0 s39012
1.0 s39170
1.0 s39241
3.0 s39364
1.0 s39631
1.0 s39804
3.0 s39805
3.0 s39825
3.0 s40597
3.0 s40623
3.0 s40862
3.0 s41520
1.0 s42113
3.0 s42221
0.0 s42778
3.0 s43114
3.0 s43683
3.0 s44620
3.0 s44824
3.0 s45972
1.0 s46066
3.0 s46846
3.0 s47677
0.0 s47691
3.0 s48631


In [100]:
in_field2=np.zeros([len(specflt),len(ids)])

for i in range(len(specflt)):
    flt_dat=fits.open(specflt[i])
    w = wcs.WCS(flt_dat[1].header)
    in_field2[i]=Source_present(w,Ra,Dec,len(flt_dat[1].data[0]),len(flt_dat[1].data))

In [101]:
totsum=sum(in_field2)

for i in range(len(ids)):
    print totsum[i],ids[i]
cat_frame

4.0 n14713
8.0 n15616
28.0 n16758
22.0 n17436
20.0 n19442
24.0 n21156
4.0 n21427
24.0 n23435
24.0 n32562
28.0 n32566
34.0 n33780
20.0 n34694
18.0 n35469
24.0 n37686
20.0 n38126
24.0 s23421
24.0 s26915
24.0 s35774
24.0 s36095
24.0 s38785
0.0 s39012
0.0 s39170
0.0 s39241
0.0 s39364
0.0 s39631
24.0 s39804
24.0 s39805
34.0 s39825
32.0 s40597
24.0 s40623
0.0 s40862
24.0 s41520
40.0 s42113
0.0 s42221
32.0 s42778
0.0 s43114
0.0 s43683
24.0 s44620
24.0 s44824
34.0 s45972
0.0 s46066
24.0 s46846
24.0 s47677
16.0 s47691
24.0 s48631


,dec,ids,ra
0,62.214608,n14713,189.236333
1,62.218809,n15616,189.238403
2,62.224840,n16758,189.162357
3,62.227726,n17436,189.171078
4,62.237460,n19442,189.123036
5,62.247548,n21156,189.239409
6,62.247344,n21427,189.368121
7,62.256571,n23435,189.338306
8,62.309192,n32562,189.168890
9,62.307577,n32566,189.161280
